In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pylab as plt
import seaborn as sns
import gc
from tqdm import tqdm
from sklearn import preprocessing
from scipy.stats import entropy, kurtosis

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth',100)
pd.set_option('display.width',100)
pd.set_option('display.max_info_rows',100)

In [2]:
import gc
gc.collect()

31

In [3]:
train_jet = pd.read_csv('E:/jet_complex_data/complex_train_R04_jet.csv')
train_particle = pd.read_feather('E:/jet_complex_data/complex_train_R04_particle.feather')
test_jet = pd.read_csv('E:/jet_complex_data/complex_test_R04_jet.csv')
test_particle = pd.read_feather('E:/jet_complex_data/complex_test_R04_particle.feather')

In [4]:
#data: jet_id级样本
data = pd.concat([train_jet, test_jet], sort=False)

In [5]:
#ans: event_id级样本
ans = data.groupby('event_id', as_index=False)['label'].agg({'label':np.mean})
ans.columns = ['event_id', 'label']
count = data.groupby('event_id', as_index=False)['jet_px'].count()
count.columns = ['event_id', 'count']
ans['count'] = count['count']
ans

,event_id,label,count
0,00000fdcf3f1454f962edf1cc6a2f64b,5.0,3
1,0000103e24d74ddd94a00a30d476a301,1.0,2
2,0000113f88864edebf628441e8b962ec,4.0,2
3,0000571f2c9b4e90a5701b32a1b2d26f,5.0,2
4,0000936dc7ee482b817758c25b567378,NaN,2
...,...,...,...
548092,fffee9ef58654385b65e3b3997cdbbd1,21.0,3
548093,fffeec406f5441ebb0904190622dd9f4,5.0,2
548094,ffff46d0d16746babfb51e1f982aea81,1.0,2
548095,ffff51e8086e4295bf86c502829bab3d,4.0,2


In [6]:
def count_all(df_group, df_merge,base_column,count_column):
    tp = df_group.groupby(base_column, as_index=False)[count_column].agg({
         base_column+'_'+count_column+'_mean': 'mean', 
         base_column+'_'+count_column+'_max': 'max', 
         base_column+'_'+count_column+'_min': 'min', 
         base_column+'_'+count_column+'_sum': 'sum', 
         base_column+'_'+count_column+'_std': 'std', 
         base_column+'_'+count_column+'_skew': 'skew', 
         base_column+'_'+count_column+'_kurt': kurtosis, 
         base_column+'_'+count_column+'_q1': lambda x: np.quantile(x, q=0.25), 
         base_column+'_'+count_column+'_q3': lambda x: np.quantile(x, q=0.75), 
    })
    df_merge = df_merge.merge(tp, on=base_column, how='left')
    return df_merge

# temp = pd.DataFrame({'1':[1,2,2,3,3,3,4,4,4,4], '2':[1,2,3,4,5,6,7,8,9,10]})
# temp
# count_all(temp, temp, '1', '2')

# particle->jet

In [7]:
#data_particle: particle_id级样本
data_particle = pd.concat([train_particle, test_particle], sort=False)

In [8]:
data_particle = pd.get_dummies(data_particle, columns=['particle_category'])

In [9]:
data_particle['particle_|p|'] = np.sqrt(np.square(data_particle['particle_px'])+np.square(data_particle['particle_py'])+np.square(data_particle['particle_pz']))
data_particle['particle_pt'] = np.sqrt(np.square(data_particle['particle_px'])+np.square(data_particle['particle_py']))

data_particle['particle_phi_cos'] = data_particle['particle_px'] / data_particle['particle_pt']
data_particle['particle_eta_sinh'] = data_particle['particle_pz'] / data_particle['particle_pt']

data_particle['particle_phi'] = np.arccos(data_particle['particle_phi_cos'])
data_particle['particle_eta'] = np.arcsinh(data_particle['particle_eta_sinh'])

for col in ['particle_phi_cos', 'particle_eta_sinh']:
    data_particle[col+'_abs'] = data_particle[col].abs()
    
data_particle['particle_energy_differ'] = data_particle['particle_energy'] - data_particle['particle_|p|']

data_particle['particle_mass/pt'] = data_particle['particle_mass'] / data_particle['particle_pt']

In [10]:
data_particle.columns.to_list()

['particle_px',
 'particle_py',
 'particle_pz',
 'particle_energy',
 'particle_mass',
 'jet_id',
 'particle_category_-2212',
 'particle_category_-2112',
 'particle_category_-321',
 'particle_category_-211',
 'particle_category_-13',
 'particle_category_-11',
 'particle_category_11',
 'particle_category_13',
 'particle_category_22',
 'particle_category_130',
 'particle_category_211',
 'particle_category_321',
 'particle_category_2112',
 'particle_category_2212',
 'particle_|p|',
 'particle_pt',
 'particle_phi_cos',
 'particle_eta_sinh',
 'particle_phi',
 'particle_eta',
 'particle_phi_cos_abs',
 'particle_eta_sinh_abs',
 'particle_energy_differ',
 'particle_mass/pt']

In [11]:
#ans_particle: jet_id级样本
ans_particle = data_particle.groupby('jet_id', as_index=False)['particle_category_-2212',
 'particle_category_-2112',
 'particle_category_-321',
 'particle_category_-211',
 'particle_category_-13',
 'particle_category_-11',
 'particle_category_11',
 'particle_category_13',
 'particle_category_22',
 'particle_category_130',
 'particle_category_211',
 'particle_category_321',
 'particle_category_2112',
 'particle_category_2212'].sum()
ans_particle

,jet_id,particle_category_-2212,particle_category_-2112,particle_category_-321,particle_category_-211,particle_category_-13,particle_category_-11,particle_category_11,particle_category_13,particle_category_22,particle_category_130,particle_category_211,particle_category_321,particle_category_2112,particle_category_2212
0,000006d8f2c143e39d40961834db96e6,0,0,1,4,0,0,0,0,13,0,2,1,0,0
1,00005b43827d451fbf43b7a6b8c90099,0,1,0,4,0,0,0,0,0,1,3,0,0,1
2,00006de576aa41ca93b2549401436baf,0,0,1,3,0,0,0,0,5,1,1,2,0,0
3,000070820a23425aa2881cc5d5041735,0,0,0,10,0,0,0,0,23,0,11,1,0,0
4,000076b478704551bc7439b81110fdfb,0,0,1,5,0,0,0,0,17,0,5,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672499,ffffbee743444ee3974e6679059d4eb5,0,0,0,4,0,0,0,0,5,2,3,0,0,0
1672500,ffffded0be154db080c3e584a51046f6,0,0,1,7,0,0,0,0,3,0,4,2,0,0
1672501,ffffe566099149af8855513c5c73621c,0,0,1,2,0,0,0,0,15,2,3,0,0,0
1672502,fffff357e4c84b61a2a471ea2753c463,1,0,0,1,0,0,0,0,8,0,3,0,1,0


In [12]:
for col in tqdm(['particle_px', 'particle_py', 'particle_pz', 'particle_energy', 'particle_mass', 
                 'particle_|p|',
                 'particle_pt',
                 'particle_phi_cos',
                 'particle_eta_sinh',
                 'particle_phi',
                 'particle_eta',
                 'particle_phi_cos_abs',
                 'particle_eta_sinh_abs',
                 'particle_energy_differ',
                 'particle_mass/pt']):
    ans_particle = count_all(data_particle, ans_particle, 'jet_id', col)

100%|███████████████████████████████████████████████████████████████████████████████| 15/15 [4:08:06<00:00, 992.41s/it]


In [13]:
data = data.merge(ans_particle, on='jet_id', how='left')
del ans_particle

# jet->event

In [14]:
gc.collect()

40

In [15]:
data['jet_energy/num'] = data['jet_energy'] / data['number_of_particles_in_this_jet']
data['jet_mass'].loc[data['jet_mass']<0] = 0
data['jet_mass/num'] = data['jet_mass'] / data['number_of_particles_in_this_jet']
data['jet_px/num'] = np.abs(data['jet_px']) / data['number_of_particles_in_this_jet']
data['jet_py/num'] = np.abs(data['jet_py']) / data['number_of_particles_in_this_jet']
data['jet_pz/num'] = np.abs(data['jet_pz']) / data['number_of_particles_in_this_jet']

D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [16]:
data['jet_|p|'] = np.sqrt(np.square(data['jet_px'])+np.square(data['jet_py'])+np.square(data['jet_pz']))
data['jet_pt'] = np.sqrt(np.square(data['jet_px'])+np.square(data['jet_py']))

data['jet_phi_cos'] = data['jet_px'] / data['jet_pt']
data['jet_eta_sinh'] = data['jet_pz'] / data['jet_pt']

data['jet_phi'] = np.arccos(data['jet_phi_cos'])
data['jet_eta'] = np.arcsinh(data['jet_eta_sinh'])

for col in ['jet_phi_cos', 'jet_eta_sinh']:
    data[col+'_abs'] = data[col].abs()
    
data['jet_energy_differ'] = data['jet_energy'] - data['jet_|p|']
data['jet_energy_differ/num'] = data['jet_energy_differ'] / data['number_of_particles_in_this_jet']

data['jet_mass/pt'] = data['jet_mass'] / data['jet_pt']

In [17]:
data_col = data.columns.to_list()
for col in data_col:
    if 'std' in col or 'skew' in col or 'kurt' in col:
        data[col] = data[col].fillna(0)
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1672504 entries, 0 to 1672503
Data columns (total 174 columns):
jet_id                                1672504 non-null object
number_of_particles_in_this_jet       1672504 non-null int64
jet_px                                1672504 non-null float64
jet_py                                1672504 non-null float64
jet_pz                                1672504 non-null float64
jet_energy                            1672504 non-null float64
jet_mass                              1672504 non-null float64
event_id                              1672504 non-null object
label                                 1134555 non-null float64
particle_category_-2212               1672504 non-null uint8
particle_category_-2112               1672504 non-null uint8
particle_category_-321                1672504 non-null uint8
particle_category_-211                1672504 non-null uint8
particle_category_-13                 1672504 non-null uint8
particle_category_

In [18]:
data_col.remove('jet_id')
data_col.remove('event_id')
data_col.remove('label')
print(len(data_col))
print(data_col)

171
['number_of_particles_in_this_jet', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_mass', 'particle_category_-2212', 'particle_category_-2112', 'particle_category_-321', 'particle_category_-211', 'particle_category_-13', 'particle_category_-11', 'particle_category_11', 'particle_category_13', 'particle_category_22', 'particle_category_130', 'particle_category_211', 'particle_category_321', 'particle_category_2112', 'particle_category_2212', 'jet_id_particle_px_mean', 'jet_id_particle_px_max', 'jet_id_particle_px_min', 'jet_id_particle_px_sum', 'jet_id_particle_px_std', 'jet_id_particle_px_skew', 'jet_id_particle_px_kurt', 'jet_id_particle_px_q1', 'jet_id_particle_px_q3', 'jet_id_particle_py_mean', 'jet_id_particle_py_max', 'jet_id_particle_py_min', 'jet_id_particle_py_sum', 'jet_id_particle_py_std', 'jet_id_particle_py_skew', 'jet_id_particle_py_kurt', 'jet_id_particle_py_q1', 'jet_id_particle_py_q3', 'jet_id_particle_pz_mean', 'jet_id_particle_pz_max', 'jet_id_particle_pz_min', 

In [19]:
for col in tqdm(data_col):
    ans = count_all(data, ans, 'event_id', col)

100%|████████████████████████████████████████████████████████████████████████████| 171/171 [14:51:04<00:00, 312.66s/it]


In [20]:
len(ans.columns.to_list())

1542

In [21]:
ans = ans.reset_index(drop=True)
ans.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548097 entries, 0 to 548096
Data columns (total 1542 columns):
event_id                                            548097 non-null object
label                                               371377 non-null float64
count                                               548097 non-null int64
event_id_number_of_particles_in_this_jet_mean       548097 non-null float64
event_id_number_of_particles_in_this_jet_max        548097 non-null int64
event_id_number_of_particles_in_this_jet_min        548097 non-null int64
event_id_number_of_particles_in_this_jet_sum        548097 non-null int64
event_id_number_of_particles_in_this_jet_std        545759 non-null float64
event_id_number_of_particles_in_this_jet_skew       318675 non-null float64
event_id_number_of_particles_in_this_jet_kurt       548097 non-null float64
event_id_number_of_particles_in_this_jet_q1         548097 non-null float64
event_id_number_of_particles_in_this_jet_q3         548097 n

In [22]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32) #if not to_feather: np.float16
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
            end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [23]:
ans_col = ans.columns.to_list()
for col in ans_col:
    if 'std' in col or 'skew' in col or 'kurt' in col:
        ans[col] = ans[col].fillna(0)
ans.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548097 entries, 0 to 548096
Data columns (total 1542 columns):
event_id                                            548097 non-null object
label                                               371377 non-null float64
count                                               548097 non-null int64
event_id_number_of_particles_in_this_jet_mean       548097 non-null float64
event_id_number_of_particles_in_this_jet_max        548097 non-null int64
event_id_number_of_particles_in_this_jet_min        548097 non-null int64
event_id_number_of_particles_in_this_jet_sum        548097 non-null int64
event_id_number_of_particles_in_this_jet_std        548097 non-null float64
event_id_number_of_particles_in_this_jet_skew       548097 non-null float64
event_id_number_of_particles_in_this_jet_kurt       548097 non-null float64
event_id_number_of_particles_in_this_jet_q1         548097 non-null float64
event_id_number_of_particles_in_this_jet_q3         548097 n

In [24]:
ans = reduce_mem_usage(ans)
ans.info(verbose=True, null_counts=True)

Mem. usage decreased to 3154.53 Mb (26.4% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548097 entries, 0 to 548096
Data columns (total 1542 columns):
event_id                                            548097 non-null object
label                                               371377 non-null float32
count                                               548097 non-null int8
event_id_number_of_particles_in_this_jet_mean       548097 non-null float32
event_id_number_of_particles_in_this_jet_max        548097 non-null int8
event_id_number_of_particles_in_this_jet_min        548097 non-null int8
event_id_number_of_particles_in_this_jet_sum        548097 non-null int16
event_id_number_of_particles_in_this_jet_std        548097 non-null float32
event_id_number_of_particles_in_this_jet_skew       548097 non-null float32
event_id_number_of_particles_in_this_jet_kurt       548097 non-null float32
event_id_number_of_particles_in_this_jet_q1         548097 non-null float32
event_id_n

In [25]:
ans.to_feather('E:/jet_complex_data/data_.feather')

In [26]:
ans.head()

,event_id,label,count,event_id_number_of_particles_in_this_jet_mean,event_id_number_of_particles_in_this_jet_max,event_id_number_of_particles_in_this_jet_min,event_id_number_of_particles_in_this_jet_sum,event_id_number_of_particles_in_this_jet_std,event_id_number_of_particles_in_this_jet_skew,event_id_number_of_particles_in_this_jet_kurt,event_id_number_of_particles_in_this_jet_q1,event_id_number_of_particles_in_this_jet_q3,event_id_jet_px_mean,event_id_jet_px_max,event_id_jet_px_min,event_id_jet_px_sum,event_id_jet_px_std,event_id_jet_px_skew,event_id_jet_px_kurt,event_id_jet_px_q1,event_id_jet_px_q3,event_id_jet_py_mean,event_id_jet_py_max,event_id_jet_py_min,event_id_jet_py_sum,event_id_jet_py_std,event_id_jet_py_skew,event_id_jet_py_kurt,event_id_jet_py_q1,event_id_jet_py_q3,event_id_jet_pz_mean,event_id_jet_pz_max,event_id_jet_pz_min,event_id_jet_pz_sum,event_id_jet_pz_std,event_id_jet_pz_skew,event_id_jet_pz_kurt,event_id_jet_pz_q1,event_id_jet_pz_q3,event_id_jet_energy_mean,event_id_jet_energy_max,event_id_jet_energy_min,event_id_jet_energy_sum,event_id_jet_energy_std,event_id_jet_energy_skew,event_id_jet_energy_kurt,event_id_jet_energy_q1,event_id_jet_energy_q3,event_id_jet_mass_mean,event_id_jet_mass_max,...,event_id_jet_eta_std,event_id_jet_eta_skew,event_id_jet_eta_kurt,event_id_jet_eta_q1,event_id_jet_eta_q3,event_id_jet_phi_cos_abs_mean,event_id_jet_phi_cos_abs_max,event_id_jet_phi_cos_abs_min,event_id_jet_phi_cos_abs_sum,event_id_jet_phi_cos_abs_std,event_id_jet_phi_cos_abs_skew,event_id_jet_phi_cos_abs_kurt,event_id_jet_phi_cos_abs_q1,event_id_jet_phi_cos_abs_q3,event_id_jet_eta_sinh_abs_mean,event_id_jet_eta_sinh_abs_max,event_id_jet_eta_sinh_abs_min,event_id_jet_eta_sinh_abs_sum,event_id_jet_eta_sinh_abs_std,event_id_jet_eta_sinh_abs_skew,event_id_jet_eta_sinh_abs_kurt,event_id_jet_eta_sinh_abs_q1,event_id_jet_eta_sinh_abs_q3,event_id_jet_energy_differ_mean,event_id_jet_energy_differ_max,event_id_jet_energy_differ_min,event_id_jet_energy_differ_sum,event_id_jet_energy_differ_std,event_id_jet_energy_differ_skew,event_id_jet_energy_differ_kurt,event_id_jet_energy_differ_q1,event_id_jet_energy_differ_q3,event_id_jet_energy_differ/num_mean,event_id_jet_energy_differ/num_max,event_id_jet_energy_differ/num_min,event_id_jet_energy_differ/num_sum,event_id_jet_energy_differ/num_std,event_id_jet_energy_differ/num_skew,event_id_jet_energy_differ/num_kurt,event_id_jet_energy_differ/num_q1,event_id_jet_energy_differ/num_q3,event_id_jet_mass/pt_mean,event_id_jet_mass/pt_max,event_id_jet_mass/pt_min,event_id_jet_mass/pt_sum,event_id_jet_mass/pt_std,event_id_jet_mass/pt_skew,event_id_jet_mass/pt_kurt,event_id_jet_mass/pt_q1,event_id_jet_mass/pt_q3
0,00000fdcf3f1454f962edf1cc6a2f64b,5.0,3,19.0,33,11,57,12.165525,1.679536,-1.5,12.00,23.00,-0.41938,79.114197,-86.136002,-1.25814,82.798431,-0.334167,-1.5,-40.186169,42.438931,-2.203573,10.598800,-9.941530,-6.61072,11.167477,1.62104,-1.5,-8.604760,1.665405,-2.127223,13.269100,-15.038000,-6.381670,14.316299,0.757725,-1.5,-9.825385,4.328165,62.311333,87.154503,18.139099,186.934006,38.353500,-1.691861,-1.5,49.889751,84.397453,7.282557,10.11920,...,0.621772,1.641775,-1.5,-0.120417,0.448451,0.822130,0.996459,0.477734,2.466390,0.298264,-1.731653,-1.5,0.734965,0.994328,0.447266,1.099838,0.053363,1.341797,0.569175,1.6227,-1.5,0.120980,0.644217,0.431006,0.589427,0.205239,1.293018,0.200753,-1.339703,-1.5,0.351795,0.543889,0.024951,0.038335,0.017861,0.074854,0.011597,1.722859,-1.5,0.018260,0.028496,0.151857,0.225544,0.112964,0.455572,0.063848,1.724023,-1.5,0.115014,0.171304
1,0000103e24d74ddd94a00a30d476a301,1.0,2,17.0,17,17,34,0.000000,0.000000,-3.0,17.00,17.00,-0.92350,152.414993,-154.261993,-1.84700,216.853394,0.000000,-2.0,-77.592751,75.745750,0.851715,3.817900,-2.114470,1.70343,4.194819,0.00000,-2.0,-0.631378,2.334807,-3.607380,2.250790,-9.465550,-7.214760,8.284703,0.000000,-2.0,-6.536465,-0.678295,153.853500,154.690002,153.016998,307.707001,1.182990,0.000000,-2.0,153.435257,154.271744,10

# analyse

In [27]:
data

,jet_id,number_of_particles_in_this_jet,jet_px,jet_py,jet_pz,jet_energy,jet_mass,event_id,label,particle_category_-2212,particle_category_-2112,particle_category_-321,particle_category_-211,particle_category_-13,particle_category_-11,particle_category_11,particle_category_13,particle_category_22,particle_category_130,particle_category_211,particle_category_321,particle_category_2112,particle_category_2212,jet_id_particle_px_mean,jet_id_particle_px_max,jet_id_particle_px_min,jet_id_particle_px_sum,jet_id_particle_px_std,jet_id_particle_px_skew,jet_id_particle_px_kurt,jet_id_particle_px_q1,jet_id_particle_px_q3,jet_id_particle_py_mean,jet_id_particle_py_max,jet_id_particle_py_min,jet_id_particle_py_sum,jet_id_particle_py_std,jet_id_particle_py_skew,jet_id_particle_py_kurt,jet_id_particle_py_q1,jet_id_particle_py_q3,jet_id_particle_pz_mean,jet_id_particle_pz_max,jet_id_particle_pz_min,jet_id_particle_pz_sum,jet_id_particle_pz_std,jet_id_particle_pz_skew,jet_id_particle_pz_kurt,jet_id_particle_pz_q1,jet_id_particle_pz_q3,...,jet_id_particle_phi_cos_abs_min,jet_id_particle_phi_cos_abs_sum,jet_id_particle_phi_cos_abs_std,jet_id_particle_phi_cos_abs_skew,jet_id_particle_phi_cos_abs_kurt,jet_id_particle_phi_cos_abs_q1,jet_id_particle_phi_cos_abs_q3,jet_id_particle_eta_sinh_abs_mean,jet_id_particle_eta_sinh_abs_max,jet_id_particle_eta_sinh_abs_min,jet_id_particle_eta_sinh_abs_sum,jet_id_particle_eta_sinh_abs_std,jet_id_particle_eta_sinh_abs_skew,jet_id_particle_eta_sinh_abs_kurt,jet_id_particle_eta_sinh_abs_q1,jet_id_particle_eta_sinh_abs_q3,jet_id_particle_energy_differ_mean,jet_id_particle_energy_differ_max,jet_id_particle_energy_differ_min,jet_id_particle_energy_differ_sum,jet_id_particle_energy_differ_std,jet_id_particle_energy_differ_skew,jet_id_particle_energy_differ_kurt,jet_id_particle_energy_differ_q1,jet_id_particle_energy_differ_q3,jet_id_particle_mass/pt_mean,jet_id_particle_mass/pt_max,jet_id_particle_mass/pt_min,jet_id_particle_mass/pt_sum,jet_id_particle_mass/pt_std,jet_id_particle_mass/pt_skew,jet_id_particle_mass/pt_kurt,jet_id_particle_mass/pt_q1,jet_id_particle_mass/pt_q3,jet_energy/num,jet_mass/num,jet_px/num,jet_py/num,jet_pz/num,jet_|p|,jet_pt,jet_phi_cos,jet_eta_sinh,jet_phi,jet_eta,jet_phi_cos_abs,jet_eta_sinh_abs,jet_energy_differ,jet_energy_differ/num,jet_mass/pt
0,17a0a1f19a7f43558cb9cca517b6e198,21,-288.5120,0.302560,1.839620,289.0660,17.78110,410e86bc0457482d9d6f81cd2d95ccf0,1.0,0,1,1,2,0,0,0,0,12,0,5,0,0,0,-13.738700,-0.092363,-146.042999,-288.512695,32.159996,-3.863304,12.105848,-7.402860,-1.121740,0.014408,0.785048,-0.610681,0.302561,0.319108,0.214523,0.353274,-0.106974,0.233851,0.087601,0.969680,-0.570297,1.839623,0.302749,1.117525,2.735379,-0.038646,0.125889,...,0.947786,20.771469,0.015469,-1.444327,0.641380,0.984116,0.999755,0.088200,0.321242,0.001227,1.852198,0.099020,0.891461,-0.565032,0.007773,0.171699,0.010053,0.136289,-0.000626,0.211117,0.030086,4.073237,13.295521,7.450581e-08,0.001644,0.056251,0.466360,0.0,1.181262,0.120566,2.643921,5.126749,0.000000,0.023525,13.765048,0.846719,13.738667,0.014408,0.087601,288.518024,288.512159,-0.999999,0.006376,3.140544,0.006376,0.999999,0.006376,0.547976,0.026094,0.061630
1,36d1005f56d343d5a09513cb7a4ead61,26,-158.4480,2.132650,-16.283400,159.7180,11.58480,6f7bf9104f8c4a619eedd129d5ce3dcb,5.0,0,1,0,6,0,0,0,0,12,0,6,0,0,1,-6.094159,-0.261038,-26.073500,-158.448120,6.783796,-2.003818,2.986048,-7.903878,-1.762480,0.082025,0.885232,-0.624319,2.132648,0.329125,0.087544,0.347457,-0.074367,0.208028,-0.626286,0.061388,-3.275600,-16.283442,0.817435,-2.305093,4.146985,-0.746367,-0.172423,...,0.945377,25.866047,0.011217,-3.825542,12.971088,0.995283,0.999735,0.101645,0.208076,0.013674,2.642768,0.048130,0.039988,-0.501092,0.073150,0.137703,0.003404,0.042799,-0.000082,0.088497,0.008839,3.979658,13.691363,3.427267e-07,0.002124,0.020773,0.128022,0.0,0.540110,0.031290,2.171960,3.995327,0.000000,0.030687,6.143000,0.445569,6.094154,0.082025,0.626285,159.296786,158.462352,-0.9999

In [28]:
ans

,event_id,label,count,event_id_number_of_particles_in_this_jet_mean,event_id_number_of_particles_in_this_jet_max,event_id_number_of_particles_in_this_jet_min,event_id_number_of_particles_in_this_jet_sum,event_id_number_of_particles_in_this_jet_std,event_id_number_of_particles_in_this_jet_skew,event_id_number_of_particles_in_this_jet_kurt,event_id_number_of_particles_in_this_jet_q1,event_id_number_of_particles_in_this_jet_q3,event_id_jet_px_mean,event_id_jet_px_max,event_id_jet_px_min,event_id_jet_px_sum,event_id_jet_px_std,event_id_jet_px_skew,event_id_jet_px_kurt,event_id_jet_px_q1,event_id_jet_px_q3,event_id_jet_py_mean,event_id_jet_py_max,event_id_jet_py_min,event_id_jet_py_sum,event_id_jet_py_std,event_id_jet_py_skew,event_id_jet_py_kurt,event_id_jet_py_q1,event_id_jet_py_q3,event_id_jet_pz_mean,event_id_jet_pz_max,event_id_jet_pz_min,event_id_jet_pz_sum,event_id_jet_pz_std,event_id_jet_pz_skew,event_id_jet_pz_kurt,event_id_jet_pz_q1,event_id_jet_pz_q3,event_id_jet_energy_mean,event_id_jet_energy_max,event_id_jet_energy_min,event_id_jet_energy_sum,event_id_jet_energy_std,event_id_jet_energy_skew,event_id_jet_energy_kurt,event_id_jet_energy_q1,event_id_jet_energy_q3,event_id_jet_mass_mean,event_id_jet_mass_max,...,event_id_jet_eta_std,event_id_jet_eta_skew,event_id_jet_eta_kurt,event_id_jet_eta_q1,event_id_jet_eta_q3,event_id_jet_phi_cos_abs_mean,event_id_jet_phi_cos_abs_max,event_id_jet_phi_cos_abs_min,event_id_jet_phi_cos_abs_sum,event_id_jet_phi_cos_abs_std,event_id_jet_phi_cos_abs_skew,event_id_jet_phi_cos_abs_kurt,event_id_jet_phi_cos_abs_q1,event_id_jet_phi_cos_abs_q3,event_id_jet_eta_sinh_abs_mean,event_id_jet_eta_sinh_abs_max,event_id_jet_eta_sinh_abs_min,event_id_jet_eta_sinh_abs_sum,event_id_jet_eta_sinh_abs_std,event_id_jet_eta_sinh_abs_skew,event_id_jet_eta_sinh_abs_kurt,event_id_jet_eta_sinh_abs_q1,event_id_jet_eta_sinh_abs_q3,event_id_jet_energy_differ_mean,event_id_jet_energy_differ_max,event_id_jet_energy_differ_min,event_id_jet_energy_differ_sum,event_id_jet_energy_differ_std,event_id_jet_energy_differ_skew,event_id_jet_energy_differ_kurt,event_id_jet_energy_differ_q1,event_id_jet_energy_differ_q3,event_id_jet_energy_differ/num_mean,event_id_jet_energy_differ/num_max,event_id_jet_energy_differ/num_min,event_id_jet_energy_differ/num_sum,event_id_jet_energy_differ/num_std,event_id_jet_energy_differ/num_skew,event_id_jet_energy_differ/num_kurt,event_id_jet_energy_differ/num_q1,event_id_jet_energy_differ/num_q3,event_id_jet_mass/pt_mean,event_id_jet_mass/pt_max,event_id_jet_mass/pt_min,event_id_jet_mass/pt_sum,event_id_jet_mass/pt_std,event_id_jet_mass/pt_skew,event_id_jet_mass/pt_kurt,event_id_jet_mass/pt_q1,event_id_jet_mass/pt_q3
0,00000fdcf3f1454f962edf1cc6a2f64b,5.0,3,19.000000,33,11,57,12.165525,1.679536,-1.5,12.00,23.00,-0.419380,79.114197,-86.136002,-1.25814,82.798431,-0.334167,-1.5,-40.186169,42.438931,-2.203573,10.598800,-9.941530,-6.610720,11.167477,1.621040,-1.5,-8.604760,1.665405,-2.127223,13.269100,-15.038000,-6.381670,14.316299,0.757725,-1.5,-9.825385,4.328165,62.311333,87.154503,18.139099,186.934006,38.353500,-1.691861,-1.5,49.889751,84.397453,7.282557,10.119200,...,0.621772,1.641775,-1.5,-0.120417,0.448451,0.822130,0.996459,0.477734,2.466390,2.982635e-01,-1.731653,-1.5,0.734965,0.994328,0.447266,1.099838,0.053363,1.341797,0.569175,1.622700,-1.5,0.120980,0.644217,0.431006,0.589427,0.205239,1.293018,0.200753,-1.339703,-1.5,0.351795,0.543889,0.024951,0.038335,0.017861,0.074854,0.011597,1.722859,-1.5,0.018260,0.028496,0.151857,0.225544,0.112964,0.455572,0.063848,1.724023,-1.5,0.115014,0.171304
1,0000103e24d74ddd94a00a30d476a301,1.0,2,17.000000,17,17,34,0.000000,0.000000,-3.0,17.00,17.00,-0.923500,152.414993,-154.261993,-1.84700,216.853394,0.000000,-2.0,-77.592751,75.745750,0.851715,3.817900,-2.114470,1.703430,4.194819,0.000000,-2.0,-0.631378,2.334807,-3.607380,2.250790,-9.465550,-7.214760,8.284703,0.000000,-2.0,-6.536465,-0.678295,153.853500,154.690002,153.016998,307.707001,1.182990,0.000000,-2.0,1